# Model 2: Tipus incidència 'Comentari problemàtic'

This notebook creates a evaluates a model to detect the issue 'Comentari problemàtic'.

It will be created a model to each language: catalan, spanish and english.

This type of issue is characterized by long coments composed for some sentences and imprecisse expressions.




In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [3]:
import spacy
from spacy.util import minibatch, compounding

# Load English tokenizer, tagger, parser, NER and word vectors
#nlp = spacy.load("ca_fasttext_wiki")
#nlp = spacy.load("es_core_news_sm")
#nlp = spacy.load("en_core_web_sm")


In [4]:
# Creates tuples with texts and cats in the whole comment

def create_texts_and_cats_comments(data_com, nlp):
    
    # Converts data frame to list
    #data_com = data_com.dropna(subset="Comentari")
    data_com = data_com[data_com["Comentari"].notnull()]
    data_list = data_com["tuples"].tolist()
    
    # Change order of comments
    random.shuffle(data_list)
    if (debug >= 2):
        print ("Tuples list:")
        print (data_list[:5])  
        
    # Split text and label of true cases into two lists
    texts, cats = zip(*data_list)
    if (debug >= 2):
        print ("Texts:")
        print (texts[0:5])
        print ("Cats:")
        print (cats[0:5])  

    return (texts, cats)

In [5]:
# Creates tuples with texts and cats, spliting sentences in the comment

def create_texts_and_cats_sentences(data_com, nlp):
    
    # Split comments in sentences
    data_com = data_com[data_com["Comentari"].notnull()]
    train_data = data_com["tuples"].tolist()
    train_list = []
    for row in train_data:
        comment = row[0]
        label = row[1]
        doc = nlp(comment)
        for sent in doc.sents:
            train_list.append((sent.text, label))
            if ((debug >=2) and (len(train_list)<=10)):
                print("+ ", sent)
                print("  ", label)
 
    if (debug >= 1):
        print("Sentences: ", len(train_list))

    # Change order of comments
    random.shuffle(train_list)
    if (debug >= 2):
        print ("Tuples list:")
        print (train_list[:5])  
        
     # Split text and label of true cases into two lists
    texts, cats = zip(*train_list)
    if (debug >= 2):
        print ("Texts:")
        print (texts[0:5])
        print ("Cats:")
        print (cats[0:5])   
        
    return (texts, cats)

In [6]:
# Loads information from preprocessed file to create de train and test data

def load_data(limit = 0, split = 0.8, language = "es", nlp = None):

    if (debug >= 1):
        print ("LOAD_DATA")
        print ("Language: ", language)
        
    # Load the model form spacy
#    nlp = spacy.load(model)
#    if (debug >= 1):
#        print ("Model loaded: ", model)
    
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print("Original data: ", data.shape[0])
        
    if (debug >= 2):
        print ("Sample:")
        display (data.sample(5))

    # Calculates label    
    data_com = data[["Comentari","TipusIncidencia"]]    # CHANGED
    if (debug >= 2):
        print ("Filtered columns:")
        display (data_com.sample(5))    

    # Calculates tuples row
    # Converts: label=True -> {"POSITIVE": True, "NEGATIVE": False}
    # label=False -> {"POSITIVE": False, "NEGATIVE": True}
    data_com["label"] = data_com["TipusIncidencia"] == "Comentari problemàtic"  # CHANGED
    data_com["tuples"] = data_com.apply(lambda row: (row["Comentari"], {"POSITIVE": bool(row["label"]), "NEGATIVE": not bool(row["label"])}), axis=1)    
    if (debug >= 2):
        print ("Tuples dataframe:")
        display (data_com.sample(5))

    # Create texts and cats lists for true cases
    data_true = data_com[data_com["label"]==True]
    if (debug >= 1):
        print ("True comments: ", data_true.shape[0])
        
    (texts_true, cats_true) = create_texts_and_cats_comments(data_true, nlp)
    #(texts_true, cats_true) = create_texts_and_cats_sentences(data_true, nlp)
    if (debug >= 2):
        print("Texts:",len(texts_true), "Cats:", len(cats_true))
        
    # Size of train_data_true and test_data_true
    split_true = int(len(texts_true) * split)+1
    if (debug >= 1):
        print ("Train True data:", split_true, ", Test True data: ", len(texts_true)-split_true)      
                
    # Create texts and cats lists for false cases        
    data_false = data_com[data_com["label"]==False]
    if (debug >= 1):
        print ("False comments:", data_false.shape[0])

    (texts_false, cats_false) = create_texts_and_cats_comments(data_false, nlp)
    #(texts_false, cats_false) = create_texts_and_cats_sentences(data_false, nlp)
    if (debug >= 2):
        print("Texts:",len(texts_false), "Cats:", len(cats_false))        
       
    # Size of train_data and test_data, considering limit
    if (limit > len(texts_true) + len(texts_false)):
        limit = len(texts_true) + len(texts_false)
    split_total = int(limit * split)

    if (debug >= 1):
        print ("Used data:", limit)
        print ("Train data: ", split_total, ", Test data: ", limit - split_total)
         
    # Size of train_data_false and test_data_false
    train_split_false = split_total - split_true
    test_split_false = (limit - split_total) - (len(texts_true) - split_true) 
        
    if (debug >= 1):
        print ("Used False comments: ", limit - len(texts_true))
        print ("Train False data: ", train_split_false, ", Test False data: ", test_split_false)  
        
         
    # Mix true and false cases and split in train and devel
    # All the true cases are included
    train_texts = texts_true[:split_true] + texts_false[:train_split_false]
    train_cats = cats_true[:split_true] + cats_false[:train_split_false]
    test_texts = texts_true[split_true:] + texts_false[-test_split_false:]
    test_cats = cats_true[split_true:] + cats_false[-test_split_false:]
        
    # Return train data and test data
    return (train_texts, train_cats), (test_texts, test_cats)

In [7]:
# Calculates the level of accuracy using the test set to compare the prediction with the label defined by the user 
# Obtains the indicators: Precission (p), Recall (r) and F-score (f)

def evaluate(tokenizer, textcat, texts, cats):
    fp_list = list()
    fn_list = list()
    
    if (debug == 2):
        print ("EVALUATE")
        
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == "NEGATIVE":
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.0
                if (debug >= 2):
                    print ("tp: ", doc)
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.0
                fp_list.append(doc)
                if (debug >= 2):
                    print ("fp: ", doc)
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
                if (debug >= 2):
                    print ("tn: ", doc)
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
                fn_list.append(doc)
                if (debug >= 2):
                    print ("fn: ", doc)   
    if (debug >= 2):
        print("tp: ", tp, ", fp: ", fp, ", tn: ", tn, ", fn: ", fn)
        
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
        
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score, 
            "tp": tp, "fp": fp, "tn": tn, "fn": fn,
            "fp_list": fp_list, "fn_list": fn_list}


In [8]:
# Loads the training information, transforms it, trains the model with this set of data, evaluate the result 
# and saves the resulting model

def train_model(model=None, language="ca", target_loss=0.001, n_texts=2000):
    if (debug >= 1):
        print ("TRAIN MODEL")
        
    # Load the model form spacy
    nlp = spacy.load(model)
    if (debug >= 1):
        print ("Model loaded: ", model)
        
    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe("textcat")

    # add label to text classifier
    textcat.add_label("POSITIVE")
    textcat.add_label("NEGATIVE")
    
    if "sentencizer" not in nlp.pipe_names:
        nlp.add_pipe(nlp.create_pipe('sentencizer'))
        
    # get names of other pipes to disable them during training
    pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec","sentencizer"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
#        if init_tok2vec is not None:
#            with init_tok2vec.open("rb") as file_:
#                textcat.model.tok2vec.from_bytes(file_.read())        
        
        if (debug >= 1):
            print ("Pipes prepared")
        
        # load training and test data
        (train_texts, train_cats), (dev_texts, dev_cats) = load_data(n_texts, 0.8, language, nlp)        

        if (debug >= 1):
            print(
                "Using {} examples ({} training, {} evaluation)".format(
                    len(train_texts)+len(dev_texts), len(train_texts), len(dev_texts)
                )
            )      

        # converts the data to the format:
        # (text, {'cats': {'POSITIVE': True, 'NEGATIVE': False}}))
        train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
        if (debug>=1):
            print ("Train_data: ")
            print (train_data[:5])        
    
        # Train the model
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}\t{:^5}".format(
            "ITER", "LOSS", "P", "R", "F","TP","FP","FN","TN"))
        batch_sizes = compounding(4.0, 32.0, 1.001)
        loss = 100
        prev_loss = 1000
        iter = 1
#        if (loss > target_loss):
        while ((loss > target_loss) | (prev_loss - loss > target_loss)) & (iter <= 20):
            prev_loss = loss
    #        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)

#            if True:
#                batch = next(batches)
            for batch in batches:
                texts, annotations = zip(*batch)
                if (debug >= 2):
                    print ("Batch: ", len(batch))
                    print (batch)

                # Train the model discarting 20% of cases to avoid generalization
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
                if (debug >= 2):
                    print ("Model updated")

            # Evaluates model and print results
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print(
                "{0:.0f}\t{1:.3f}\t{2:.3f}\t{3:.3f}\t{4:.3f}\t {5:.0f}\t {6:.0f}\t {7:.0f}\t {8:.0f}".format(  # print a simple table
                    iter,
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                    scores["tp"],
                    scores["fp"],
                    scores["fn"],
                    scores["tn"]
                )
            )  
            loss = losses["textcat"]
            iter = iter + 1
            
        print("False positives:")
        display(scores["fp_list"][:10])
        print("False negatives:")
        display(scores["fn_list"][:5])
        
        print("Model trained")    
    
    # test the trained model
    print ("")
    print ("Some examples: ")
    test_texts ={"ca": ["Costa que pengi els materials acordats al campus",
                    "No he tingut aquesta professora",
                    "Sembla que tingui ganes de que patim, i ens tracta de tontos",
                    "No ha corregit examens fets a octubre, i estem al mes de gener.",
                    "No acabem d'entendre què és el que vol, però sembla que no va amb ella el tema"],
            "es": ["No he aprendido nada en ninguna de sus clases, ni siquiera he encontrado herramientas para buscarme la vida.", 
                   "Fue una de las maestras más sensatas y preocupadas por los estudiantes",
                   "Considero que no debería de seguir siendo profesor del inefc, puesto que ni tiene conocimientos ni ganas.",
                   "Hace que los alumnos vayan con muchas ganas a sus clases"
                   "Habla mal a los alumnos y se reprime a los que se quejan"
                  ],
            "en": ["I haven't had this professor",
                   "We don't know this person",
                   "He isn't a good professor",
                   "We had only a few lessons"
                  ]
        }     
    for test_text in test_texts[language]:
        doc = nlp(test_text)
        print(test_text, doc.cats)
    print ("")
    
    # Save de model
    #with nlp.use_params(optimizer.averages):
    #    nlp.to_disk(pathmodel + language)
    #print("Saved model to", pathmodel + language)
        
    # test the saved model
    #print("Loading from", pathmodel+ language)
    #nlp2 = spacy.load(pathmodel +  language)
    #for test_text in test_texts[language]:
    #    doc2 = nlp2(test_text)
    #    print(test_text, doc2.cats)
    

In [9]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/model2/"
debug = 1

In [15]:
debug = 1
target_loss = 0.001
n_texts = 700
#languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", "en":"en_core_web_sm" }
#languages = {"ca":"ca_fasttext_wiki_lg", "es":"es_core_news_lg", "en":"en_core_web_sm" }
languages = {"ca":"ca_fasttext_wiki_lg"}

for language in languages:
    model = languages[language]
    
    print()
    #print ("TRAINING LANGUAGE: " + language)    
    #print ("Model: " + model)
    train_model (model, language, target_loss, n_texts)



TRAIN MODEL
Model loaded:  ca_fasttext_wiki_lg
Pipes prepared
LOAD_DATA
Language:  ca
Original data:  6975
True comments:  387
Train True data: 310 , Test True data:  77
False comments: 6588
Used data: 700
Train data:  560 , Test data:  140
Used False comments:  313
Train False data:  250 , Test False data:  63
Using 700 examples (560 training, 140 evaluation)
Train_data: 
[('Allarga molt la durada de les classes', {'cats': {'POSITIVE': True, 'NEGATIVE': False}}), ('La seva actitud en vers a la Gloria es de "pilota" total, ja que diu que sí a tot sense saber el que ha dit ni com ho ha dit. Les seves paraules tremoles quan la Gloria està escoltant perquè sap que aquesta assignatura gira en el món de la Gloria', {'cats': {'POSITIVE': True, 'NEGATIVE': False}}), (" és un xulo. Motius: en una videoconferencia va menjar xicle tota l'estona, i en un moment en el qual cada grup explicava lo que havia trobat sobre una feina, ell es va aixecar, es va treure els auriculars i va començar a cuina

<ipython-input-6-6b79bd23b264>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_com["label"] = data_com["TipusIncidencia"] == "Comentari problemàtic"  # CHANGED
<ipython-input-6-6b79bd23b264>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_com["tuples"] = data_com.apply(lambda row: (row["Comentari"], {"POSITIVE": bool(row["label"]), "NEGATIVE": not bool(row["label"])}), axis=1)


1	3.758	0.762	0.831	0.795	 64	 20	 13	 43
2	1.683	0.791	0.883	0.834	 68	 18	 9	 45
3	0.407	0.753	0.909	0.824	 70	 23	 7	 40
4	0.140	0.744	0.870	0.802	 67	 23	 10	 40
5	0.048	0.759	0.857	0.805	 66	 21	 11	 42
6	0.018	0.750	0.857	0.800	 66	 22	 11	 41
7	0.014	0.756	0.883	0.814	 68	 22	 9	 41
8	0.008	0.747	0.883	0.810	 68	 23	 9	 40
9	0.001	0.744	0.870	0.802	 67	 23	 10	 40
10	0.002	0.753	0.870	0.807	 67	 22	 10	 41
11	0.000	0.753	0.870	0.807	 67	 22	 10	 41
12	0.000	0.753	0.870	0.807	 67	 22	 10	 41
False positives:


[Bon professor tècnicament parlant.,
 Literalment ens hem estudiat la seva part del temari a casa sols.,
 Potser fa falta més planificació sobre els temes a abordar. Tot i així els temes abordats són d'utilitat i l'ensenyament és bo. Em reitero amb la necessitat tocar altres exemples d'altres territoris de muntanya fora de Catalunya (resta de Pirineu, Euskal Herria, França, Alps, Asturies, Escandinàvia, etc..).,
 Que un professor utilitzi els apunts d’un altre professor, sense haver-los revisat es molt greu. Estan plens d’errors, que any rere any es repeteixen. Canviar arxius sense dir res i després trobar-nos amb 2 o 3 versions dels apunts amb informació diferent entre els companys complica molt l’estudi.
 Ficar una nota mínima de 5 en un examen complica bastant les coses ja que en mal dia en l’examen t’avoca a la recuperació sense cap altra possibilitat. 
 Trobo correcte realitzar un treball en Cype però en cap cas i mai fer-lo en horari fora de classe i de 19-21 hores. No es pot exp

False negatives:


[En un parell d'ocasions vam sortir a les 20.00 de classe quan havíem de sortir a les 19.00. No estic d'acord amb què ens hagin de treure temps de la nostra vida fent coses que realment podria haver donat temps de fer dins l'horari establert.,
 Un aspecte important de millora es la manera d'adreçar-se als alumnes, ja que en ocasions no ha estat del tot correcte (missatges del campus i correus).,
 La metodologia de l'assignatura de comunicació lingüística no ha estat gens ben enfocada. A més a més, salta a la primera com si fòssim nens i nenes de primària. I trobo que hi ha agut alguna falta de respecta o mala paraula/actitud cap a nosaltres. ,
 Aquest home primer de tot es passa les primeres classes explicant com funciona l'assignatura perquè és tan complicat d'entendre que ha de dedicar les primeres setmanes només a explicar el funcionament de l'assignatura. I amb això ja perdem temps d'aprenentatge. Es pensa que el funcionament és perfecte i innovador però la veritat és que tal i com

Model trained

Some examples: 
Costa que pengi els materials acordats al campus {'POSITIVE': 0.9999992847442627, 'NEGATIVE': 6.741091169715219e-07}
No he tingut aquesta professora {'POSITIVE': 1.0, 'NEGATIVE': 2.7282121450866725e-08}
Sembla que tingui ganes de que patim, i ens tracta de tontos {'POSITIVE': 0.9999921321868896, 'NEGATIVE': 7.87929275247734e-06}
No ha corregit examens fets a octubre, i estem al mes de gener. {'POSITIVE': 0.9999998807907104, 'NEGATIVE': 1.3025432110680413e-07}
No acabem d'entendre què és el que vol, però sembla que no va amb ella el tema {'POSITIVE': 1.0, 'NEGATIVE': 1.1511594344426612e-08}



## Test load_data function
debug = 2
n_texts = 2000
language = "ca"
model = "ca_fasttext_wiki"

# Load the model form spacy
nlp = spacy.load(model)
if (debug >= 1):
    print ("Model loaded: ", model)

# load training and test data
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(n_texts, 0.8, language, nlp)        
